# SET Alll Securities

Get the ticker list of all listed company from SET website


สินค้า ->ตราสารทุน  https://www.set.or.th/th/products/equities/equities_p1.html
    
ระบบซื้อขายหลักทรัพย์    https://www.set.or.th/th/products/trading/equity/trading_p1.html

ข้อมูลบริษัท/หลักทรัพย์ -> ตราสารทุน  -> รายชื่อบริษัท / หลักทรัพย์

https://www.set.or.th/th/company/companylist.html



รายละเอียด
รายชื่อบริษัทจดทะเบียนในตลาดหลักทรัพย์ (ปรับปรุงข้อมูลทุกวันทำการแรกของแต่ละสัปดาห์)
https://www.set.or.th/dat/eod/listedcompany/static/listedCompanies_th_TH.xls




Output of this script is 'stock_tickers.csv' which is filtered to only tickers in SET exchange.

In [19]:
import requests

src = "https://www.set.or.th/dat/eod/listedcompany/static/listedCompanies_th_TH.xls"
r = requests.get(src)
r.status_code

200

In [2]:
r.encoding

In [5]:
content = r.content.decode('tis-620')
content[:300] # sample first 300 characters

'<table cellSpacing="0" cellPadding="0" width="100%" border="0">\n  <tr>\n    <td colspan="6" align="left"><strong>รายชื่อบริษัทจดทะเบียนในตลาดหลักทรัพย์แห่งประเทศไทย</strong></td>\n    <td colspan="4" align="right"><strong>ข้อมูล ณ วันที่ 07 เม.ย. 2563<strong></td>\n  </tr>\n  <tr align="center" bgcolor='


Try other tools to read Excel.
~~~
!pip install openpyxl
!pip install lxml
~~~

Save output to **securities.xls**.


In [5]:
import pandas as pd

outFile = "securities.xls"

# Reference: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
with open(outFile, "w", encoding='utf-8') as f:
    f.write(content)



b'<html><body>\n<table cellspacing="0" cellpadding="0" width="100%" border="0">\n  <tr>\n    <td colspan="6" align="left"><strong>&#3619;&#3634;&#3618;&#3594;&#3639;&#3656;&#3629;&#3610;&#3619;&#3636;&#3625;&#3633;&#3607;&#3592;&#3604;&#3607;&#3632;&#3648;&#3610;&#3637;&#3618;&#3609;&#3651;&#3609;&#3605;&#3621;&#3634;&#3604;&#3627;&#3621;&#3633;&#3585;&#3607;&#3619;&#3633;&#3614;&#3618;&#3660;&#3649;&#3627;&#3656;&#3591;&#3611;&#3619;&#3632;&#3648;&#3607;&#3624;&#3652;&#3607;&#3618;</strong></td>\n    <td colspan="4" align="right"><strong>&#3586;&#3657;&#3629;&#3617;&#3641;&#3621; &#3603; &#3623;&#3633;&#3609;&#3607;&#3637;&#3656; 09 &#3617;&#3637;.&#3588;. 2563<strong></strong></strong></td>\n  </tr>\n  <tr align="center" bgcolor="#C0C0C0"> \n    <td nowrap><strong>&#3627;&#3621;&#3633;&#3585;&#3607;&#3619;&#3633;&#3614;&#3618;&#3660;</strong></td>\n    <td nowrap><strong>&#3610;&#3619;&#3636;&#3625;&#3633;&#3607;</strong></td>\n    <td nowrap><strong>&#3605;&#3621;&#3634;&#3604;</str

## Parse the Data

Parse data downloaded from the website. The file is in HTML instead of Excel.

In [6]:
from lxml import etree
html = etree.HTML(r.content.decode('tis-620')) # above step show encoding is TIS-620

result = etree.tostring(html, pretty_print=True, method="html")

print(result)

b'<html><body>\n<table cellspacing="0" cellpadding="0" width="100%" border="0">\n  <tr>\n    <td colspan="6" align="left"><strong>&#3619;&#3634;&#3618;&#3594;&#3639;&#3656;&#3629;&#3610;&#3619;&#3636;&#3625;&#3633;&#3607;&#3592;&#3604;&#3607;&#3632;&#3648;&#3610;&#3637;&#3618;&#3609;&#3651;&#3609;&#3605;&#3621;&#3634;&#3604;&#3627;&#3621;&#3633;&#3585;&#3607;&#3619;&#3633;&#3614;&#3618;&#3660;&#3649;&#3627;&#3656;&#3591;&#3611;&#3619;&#3632;&#3648;&#3607;&#3624;&#3652;&#3607;&#3618;</strong></td>\n    <td colspan="4" align="right"><strong>&#3586;&#3657;&#3629;&#3617;&#3641;&#3621; &#3603; &#3623;&#3633;&#3609;&#3607;&#3637;&#3656; 07 &#3648;&#3617;.&#3618;. 2563<strong></strong></strong></td>\n  </tr>\n  <tr align="center" bgcolor="#C0C0C0"> \n    <td nowrap><strong>&#3627;&#3621;&#3633;&#3585;&#3607;&#3619;&#3633;&#3614;&#3618;&#3660;</strong></td>\n    <td nowrap><strong>&#3610;&#3619;&#3636;&#3625;&#3633;&#3607;</strong></td>\n    <td nowrap><strong>&#3605;&#3621;&#3634;&#3604;</str

In [7]:
stocks = {}

class Company:
    def __init__(self, row):
        self.exchange = row.getchildren()[2].text.strip().upper()
        self.symbol = row.getchildren()[0].text.strip().upper()
        self.name = row.getchildren()[1].text.strip()
        self.industry = row.getchildren()[3].text.strip()
        self.sector = row.getchildren()[4].text.strip()
        self.address = row.getchildren()[5].text.strip()
        self.zipcode = row.getchildren()[6].text.strip()
        self.telephone = row.getchildren()[7].text.strip()
        self.fax = row.getchildren()[8].text.strip()
        self.zipcode = row.getchildren()[9].text.strip()
        self._row = row
        
    def __str__(self):
        return """<Company %s:%s >""" % (self.exchange, self.symbol,)

for row in html.xpath('//tr'):
    print(len(row.getchildren()))
    if (len(row.getchildren()) == 10) and (row.getchildren()[0].text is not None):
        for c in row.getchildren():
            print(c.text)
        try:
            company = Company(row)
            print(company)
            stocks[company.symbol] = company
        except Exception as e:
            print("Error for %s %s" % (str(e), row))

2
10
10
2S                  
บริษัท 2 เอส เมทัล จำกัด (มหาชน)
mai     
สินค้าอุตสาหกรรม 
-
8/5 หมู่ที่ 14 ตำบลท่าช้าง อำเภอบางกล่ำ สงขลา
90110
0-7480-0111
0-7480-1286
www.ss.co.th
<Company MAI:2S >
10
7UP                 
บริษัท เซเว่น ยูทิลิตี้ส์ แอนด์ พาวเวอร์ จำกัด (มหาชน)
SET     
ทรัพยากร
พลังงานและสาธารณูปโภค
73 อาคารมหาชล ซอยสุขุมวิท 62  ถนนสุขุมวิท แขวงพระโขนงใต้ เขตพระโขนง กทม.
10260
0-2741-5700
0-2741-5724
www.sevenup.co.th
<Company SET:7UP >
10
A                   
บริษัท อารียา พรอพเพอร์ตี้ จำกัด (มหาชน)
SET     
อสังหาริมทรัพย์และก่อสร้าง
พัฒนาอสังหาริมทรัพย์
999 ถนนประดิษฐ์มนูธรรม แขวงสะพานสอง เขตวังทองหลาง กทม.
10310
0-2798-9999, 1797
0-2798-9990
http://www.areeya.co.th
<Company SET:A >
10
A5                  
บริษัท แอสเซท ไฟว์ กรุ๊ป จำกัด (มหาชน)
mai     
 บริการ
-
เลขที่ 349 อาคารเอสเจ อินฟินิท วัน บิสซิเนส คอมเพล็กซ์ ชั้น ที่ 19 ยูนิต 1901 ถนนวิภาวดีรังสิต แขวงจอมพล เขตจตุจักร กทม.
10900
0-2026-3512
0-2026-3513
www.assetfive.co.th
<Company MAI:A5 >
10
AAV            

<Company SET:BWG >
10
B-WORK              
ทรัสต์เพื่อการลงทุนในสิทธิการเช่าอสังหาริมทรัพย์บัวหลวง ออฟฟิศ
SET     
อสังหาริมทรัพย์และก่อสร้าง
กองทุนรวมอสังหาริมทรัพย์และกองทรัสต์เพื่อการลงทุนในอสังหาริมทรัพย์
เลขที่ 175 อาคารสาธรซิตี้ทาวเวอร์ ชั้น 7 ชั้น 21 และ ชั้น 26 ถนนสาทรใต้ แขวงทุ่งมหาเมฆ เขตสาทร กทม.
10120
0-2674-6488
0-2679-5996
http://www.bworkreit.com
<Company SET:B-WORK >
10
CAZ                 
บริษัท ซี เอ แซด (ประเทศไทย) จำกัด (มหาชน)
mai     
อสังหาริมทรัพย์และก่อสร้าง
-
เลขที่ 239 ถนนห้วยโป่ง-หนองบอน ตำบลห้วยโป่ง อำเภอเมืองระยอง ระยอง
21150
0-3860-6242
0-3860-6166
www.caz.co.th
<Company MAI:CAZ >
10
CBG                 
บริษัท คาราบาวกรุ๊ป จำกัด (มหาชน)
SET     
เกษตรและอุตสาหกรรมอาหาร
อาหารและเครื่องดื่ม
393 อาคาร 393 สีลม ชั้น 7-10 ถนนสีลม แขวงสีลม เขตบางรัก กทม.
10500
0-2636-6111
0-2636-7949
www.carabaogroup.com
<Company SET:CBG >
10
CCET                
บริษัท แคล-คอมพ์ อีเล็คโทรนิคส์ (ประเทศไทย) จำกัด (มหาชน)
SET     
เทคโนโลยี
ชิ้นส่วนอิเล็กทรอนิกส์
อาคารซีทีไอ ทา

10
IMH                 
บริษัท โรงพยาบาลอินเตอร์เมดิคัล แคร์ แอนด์ แล็บ จำกัด (มหาชน)
mai     
 บริการ
-
442 ถนนบางแวก แขวงบางแวก เขตภาษีเจริญ กทม.
10160
0-2865-0044-49
0-2410-4284
http://www.intermedthai.com/
<Company MAI:IMH >
10
IMPACT              
ทรัสต์เพื่อการลงทุนในอสังหาริมทรัพย์อิมแพ็คโกรท
SET     
อสังหาริมทรัพย์และก่อสร้าง
กองทุนรวมอสังหาริมทรัพย์และกองทรัสต์เพื่อการลงทุนในอสังหาริมทรัพย์
เลขที่ 47/569-576 หมู่ที่ 3 ชั้น 10 คอนโดมิเนียมอุตสาหกรรม นิวเจนีวา ถนนป๊อปปูล่า 3 ตำบลบ้านใหม่ อำเภอปากเกร็ด นนทบุรี
11120
0-2833-4455
0-2833-4456
www.impactgrowthreit.com
<Company SET:IMPACT >
10
INET                
บริษัท อินเทอร์เน็ตประเทศไทย จำกัด (มหาชน)
SET     
เทคโนโลยี
เทคโนโลยีสารสนเทศและการสื่อสาร
อาคารไทยซัมมิท ทาวเวอร์ ชั้น 10-12 และชั้น IT, 1768 ถนนเพชรบุรีตัดใหม่ แขวงบางกะปิ เขตห้วยขวาง กทม.
10310
0-2257-7000
0-2257-7222
www.inet.co.th
<Company SET:INET >
10
INGRS               
บริษัท อิงเกรส อินดัสเตรียล (ไทยแลนด์) จำกัด (มหาชน)
SET     
สินค้าอุตสาหกรรม
ยานยนต์
เลขที่ 

12 หมู่บ้านประชานิเวศน์ 1 ถนนเทศบาลนฤมาล แขวงลาดยาว เขตจตุจักร กทม.
10900
0-2589-0020
0-2589-7049
www.matichon.co.th
<Company SET:MATI >
10
MAX                 
บริษัท แมกซ์ เมทัล คอร์ปอเรชั่น จำกัด (มหาชน)
SET     
สินค้าอุตสาหกรรม
เหล็ก
เลขที่ 1077/4-6 อาคารไซมิส ราชครู ชั้น 2 ถนนพหลโยธิน แขวงสามเสนใน เขตพญาไท กทม.
10400
0-2357-1377-80
0-2357-1380
www.maxmetalcorp.co.th
<Company SET:MAX >
10
MBAX                
บริษัท มัลติแบกซ์ จำกัด (มหาชน)
mai     
สินค้าอุตสาหกรรม 
-
211 หมู่ที่ 3 ตำบลทุ่งสุขลา อำเภอศรีราชา ชลบุรี
20230
038-491-725-9
038-492-485
www.multibax.com
<Company MAI:MBAX >
10
MBK                 
บริษัท เอ็ม บี เค จำกัด (มหาชน)
SET     
อสังหาริมทรัพย์และก่อสร้าง
พัฒนาอสังหาริมทรัพย์
อาคารเอ็มบีเคเซ็นเตอร์ ชั้น 8, 444 ถนนพญาไท แขวงวังใหม่ เขตปทุมวัน กทม.
10330
0-2853-9000
0-2853-7000
www.mbkgroup.co.th
<Company SET:MBK >
10
MBKET               
บริษัทหลักทรัพย์ เมย์แบงก์ กิมเอ็ง (ประเทศไทย) จำกัด (มหาชน)
SET     
ธุรกิจการเงิน
เงินทุนและหลักทรัพย์
เลขที่ 999/9 อาคาร ดิ 

10
PATO                
บริษัท พาโตเคมีอุตสาหกรรม จำกัด (มหาชน)
SET     
สินค้าอุตสาหกรรม
ปิโตรเคมีและเคมีภัณฑ์
3388 ถนนเพชรบุรีตัดใหม่ แขวงบางกะปิ เขตห้วยขวาง กทม.
10310
0-2318-5612-19, 0-2318-0360-9
0-2318-0367
www.patochemical.com
<Company SET:PATO >
10
PB                  
บริษัท เพรซิเดนท์ เบเกอรี่ จำกัด (มหาชน)
SET     
เกษตรและอุตสาหกรรมอาหาร
อาหารและเครื่องดื่ม
อาคารชุด อาร์ เอส ทาวเวอร์ ชั้น 29, 121/84-85 ถนนรัชดาภิเษก แขวงดินแดง เขตดินแดง กทม.
10400
02-209-3000
02-209-3091
http://www.farmhouse.co.th
<Company SET:PB >
10
PCSGH               
บริษัท พี.ซี.เอส.แมชีน กรุ๊ปโฮลดิ้ง จำกัด (มหาชน)
SET     
สินค้าอุตสาหกรรม
ยานยนต์
เลขที่ 2/1-9 หมู่ 3 ตำบลโคกกรวด อำเภอเมืองนครราชสีมา นครราชสีมา
30280
0-4470-1300
0-4470-1399
www.pcsgh.com
<Company SET:PCSGH >
10
PDG                 
บริษัท พรอดดิจิ จำกัด (มหาชน)
mai     
สินค้าอุตสาหกรรม 
-
7/3 หมู่ที่ 3 ตำบลบางกระเบา อำเภอนครชัยศรี นครปฐม
73120
0-3433-2611-2
0-3433-2613
www.prodigy.co.th
<Company MAI:PDG >
10
PDI                 
บริษ

ธุรกิจการเงิน
ธนาคาร
9 ถนนรัชดาภิเษก แขวงจตุจักร เขตจตุจักร กทม.
10900
0-2544-1111, 0-2777-7777
0-2937-7931
www.scb.co.th
<Company SET:SCB >
10
SCC                 
บริษัท ปูนซิเมนต์ไทย จำกัด(มหาชน)
SET     
อสังหาริมทรัพย์และก่อสร้าง
วัสดุก่อสร้าง
1 ถนนปูนซิเมนต์ไทย แขวงบางซื่อ เขตบางซื่อ กทม.
10800
0-2586-3333, 0-2586-4444
0-2587-2199, 0-2587-2201
www.scg.com
<Company SET:SCC >
10
SCCC                
บริษัท ปูนซีเมนต์นครหลวง จำกัด (มหาชน)
SET     
อสังหาริมทรัพย์และก่อสร้าง
วัสดุก่อสร้าง
199 อาคารคอลัมน์ทาวเวอร์ ชั้น ที่ 3, 10 - 12 ถนนรัชดาภิเษก แขวงคลองเตย เขตคลองเตย กทม.
10110
0-2797-7000
0-2797-7001-2
http://www.siamcitycement.com
<Company SET:SCCC >
10
SCG                 
บริษัท สหโคเจน (ชลบุรี) จำกัด (มหาชน)
SET     
ทรัพยากร
พลังงานและสาธารณูปโภค
636 หมู่ 11 ถนนสุขาภิบาล 8 ตำบลหนองขาม  อำเภอศรีราชา ชลบุรี
20230
0-3848-1552-5
0-3848-1551
www.sahacogen.com
<Company SET:SCG >
10
SCI                 
บริษัท เอสซีไอ อีเลคตริค จำกัด (มหาชน)
SET     
ทรัพยากร
พลังงานและสาธารณูปโภค
1

<Company SET:SSSC >
10
SST                 
บริษัท ทรัพย์ศรีไทย จำกัด (มหาชน)
SET     
เกษตรและอุตสาหกรรมอาหาร
อาหารและเครื่องดื่ม
2044/25-27 ถนนเพชรบุรีตัดใหม่ แขวงบางกะปิ เขตห้วยขวาง กทม.
10310
0-2318-5514-6
0-2318-3490
http://www.subsrithai.co.th
<Company SET:SST >
10
SSTRT               
ทรัสต์เพื่อการลงทุนในอสังหาริมทรัพย์ทรัพย์ศรีไทย
SET     
อสังหาริมทรัพย์และก่อสร้าง
กองทุนรวมอสังหาริมทรัพย์และกองทรัสต์เพื่อการลงทุนในอสังหาริมทรัพย์
2044/25-27 ถนนเพชรบุรีตัดใหม่ แขวงบางกะปิ เขตห้วยขวาง กทม.
10310
0-2318-5514-5
0-2318-3490-1
www.sstrm.co.th
<Company SET:SSTRT >
10
STA                 
บริษัท ศรีตรังแอโกรอินดัสทรี จำกัด (มหาชน)
SET     
เกษตรและอุตสาหกรรมอาหาร
ธุรกิจการเกษตร
10 ซอย 10 ถนนเพชรเกษม ตำบลหาดใหญ่ อำเภอหาดใหญ่ สงขลา
90110
0-7434-4663
0-7434-4676, 0-7434-4677, 0-7423-7423
www.sritranggroup.com
<Company SET:STA >
10
STANLY              
บริษัท ไทยสแตนเลย์การไฟฟ้า จำกัด (มหาชน)
SET     
สินค้าอุตสาหกรรม
ยานยนต์
29/3 หมู่ 1 ถนนบางพูน-รังสิต ตำบลบ้านกลาง อำเภอเมือง ปทุมธานี

mai     
สินค้าอุตสาหกรรม 
-
164 หมู่ 5 ซอยเทศบาล 55 ถนนสุขุมวิท ตำบลท้ายบ้านใหม่ อำเภอเมืองสมุทรปราการ สมุทรปราการ
10280
0-2769-7300
0-2769-7349
www.yuasathai.com
<Company MAI:YUASA >
10
ZEN                 
บริษัท เซ็น คอร์ปอเรชั่น กรุ๊ป จำกัด (มหาชน)
SET     
เกษตรและอุตสาหกรรมอาหาร
อาหารและเครื่องดื่ม
เลขที่ 662 ซอยอ่อนนุช 17 แขวงสวนหลวง เขตสวนหลวง กทม.
10250
0-2019-5000 ต่อ 982
0-2030-5322
www.zengroup.co.th
<Company SET:ZEN >
10
ZIGA                
บริษัท ซิก้า อินโนเวชั่น จำกัด (มหาชน)
mai     
สินค้าอุตสาหกรรม 
-
999/9, 10, 11 หมู่ที่ 9 ต.ในคลองบางปลากด อ.พระสมุทรเจดีย์ สมุทรปราการ
10290
0-2816-9315-6
0-2816-9315-6 ต่อ 444
www.ziga.co.th
<Company MAI:ZIGA >
10
ZMICO               
บริษัทหลักทรัพย์ ซีมิโก้ จำกัด (มหาชน)
SET     
ธุรกิจการเงิน
เงินทุนและหลักทรัพย์
เลขที่ 287 อาคารลิเบอร์ตี้สแควร์ ชั้น 18 ถนนสีลม แขวงสีลม เขตบางรัก กทม.
10500
0-2695-5000
0-2631-1709
www.seamico.com
<Company SET:ZMICO >


## Count total ticker symbols


In [12]:
len(stocks)

731

## Save output to Excel

Use pandas to save the output to excel file.

In [13]:
import pandas as pd

In [15]:
# idx = ['symbol', ]
# pd.DataFrame(stocks, index=idx)

all_stocks = {}
for symbol, company in stocks.items():
  all_stocks[symbol] = {
        'symbol': company.symbol,
        'exchange': company.exchange,
        'industry': company.industry,
        'sector': company.sector,
        'name': company.name
  }  



In [16]:
all_stocks

{'2S': {'symbol': '2S',
  'exchange': 'MAI',
  'industry': 'สินค้าอุตสาหกรรม',
  'sector': '-',
  'name': 'บริษัท 2 เอส เมทัล จำกัด (มหาชน)'},
 '7UP': {'symbol': '7UP',
  'exchange': 'SET',
  'industry': 'ทรัพยากร',
  'sector': 'พลังงานและสาธารณูปโภค',
  'name': 'บริษัท เซเว่น ยูทิลิตี้ส์ แอนด์ พาวเวอร์ จำกัด (มหาชน)'},
 'A': {'symbol': 'A',
  'exchange': 'SET',
  'industry': 'อสังหาริมทรัพย์และก่อสร้าง',
  'sector': 'พัฒนาอสังหาริมทรัพย์',
  'name': 'บริษัท อารียา พรอพเพอร์ตี้ จำกัด (มหาชน)'},
 'A5': {'symbol': 'A5',
  'exchange': 'MAI',
  'industry': 'บริการ',
  'sector': '-',
  'name': 'บริษัท แอสเซท ไฟว์ กรุ๊ป จำกัด (มหาชน)'},
 'AAV': {'symbol': 'AAV',
  'exchange': 'SET',
  'industry': 'บริการ',
  'sector': 'ขนส่งและโลจิสติกส์',
  'name': 'บริษัท เอเชีย เอวิเอชั่น จำกัด (มหาชน)'},
 'ABICO': {'symbol': 'ABICO',
  'exchange': 'MAI',
  'industry': 'เกษตรและอุตสาหกรรมอาหาร',
  'sector': '-',
  'name': 'บริษัท เอบิโก้ โฮลดิ้งส์ จำกัด (มหาชน)'},
 'ABM': {'symbol': 'ABM',
  'exchange': '

In [17]:
all_symbols = pd.DataFrame(all_stocks)

all_symbols

,2S,7UP,A,A5,AAV,ABICO,ABM,ABPIF,ACAP,ACC,...,WORLD,WP,WPH,XO,YCI,YGG,YUASA,ZEN,ZIGA,ZMICO
symbol,2S,7UP,A,A5,AAV,ABICO,ABM,ABPIF,ACAP,ACC,...,WORLD,WP,WPH,XO,YCI,YGG,YUASA,ZEN,ZIGA,ZMICO
exchange,MAI,SET,SET,MAI,SET,MAI,MAI,SET,MAI,SET,...,SET,SET,SET,MAI,SET,MAI,MAI,SET,MAI,SET
industry,สินค้าอุตสาหกรรม,ทรัพยากร,อสังหาริมทรัพย์และก่อสร้าง,บริการ,บริการ,เกษตรและอุตสาหกรรมอาหาร,ทรัพยากร,ทรัพยากร,ธุรกิจการเงิน,สินค้าอุปโภคบริโภค,...,-,ทรัพยากร,บริการ,เกษตรและอุตสาหกรรมอาหาร,สินค้าอุตสาหกรรม,บริการ,สินค้าอุตสาหกรรม,เกษตรและอุตสาหกรรมอาหาร,สินค้าอุตสาหกรรม,ธุรกิจการเงิน
sector,-,พลังงานและสาธารณูปโภค,พัฒนาอสังหาริมทรัพย์,-,ขนส่งและโลจิสติกส์,-,-,พลังงานและสาธารณูปโภค,-,ของใช้ในครัวเรือนและสำนักงาน,...,หมวดบริษัทจดทะเบียนที่อยู่ระหว่างฟื้นฟูการดำเน...,พลังงานและสาธารณูปโภค,การแพทย์,-,ปิโตรเคมีและเคมีภัณฑ์,-,-,อาหารและเครื่องดื่ม,-,เงินทุนและหลักทรัพย์
name,บริษัท 2 เอส เมทัล จำกัด (มหาชน),บริษัท เซเว่น ยูทิลิตี้ส์ แอนด์ พาวเวอร์ จำกัด...,บริษัท อารียา พรอพเพอร์ตี้ จำกัด (มหาชน),บริษัท แอสเซท ไฟว์ กรุ๊ป จำกัด (มหาชน),บริษัท เอเชีย เอวิเอชั่น จำกัด (มหาชน),บริษัท เอบิโก้ โฮลดิ้งส์ จำกัด (มหาชน),บริษัท เอเชีย ไบโอแมส จำกัด (มหาชน),กองทุนรวมโครงสร้างพื้นฐานโรงไฟฟ้า อมตะ บี.กริม...,บริษัท เอเชีย แคปปิตอล กรุ๊ป จำกัด (มหาชน),บริษัท แอดวานซ์ คอนเนคชั่น คอร์ปอเรชั่น จำกัด ...,...,บริษัท เวิลด์ คอร์ปอเรชั่น จำกัด (มหาชน),บริษัท ดับบลิวพี เอ็นเนอร์ยี่ จำกัด (มหาชน),บริษัท โรงพยาบาลวัฒนแพทย์ ตรัง จำกัด (มหาชน),บริษัท เอ็กโซติค ฟู้ด จำกัด (มหาชน),บริษัท ยงไทย จำกัด (มหาชน),บริษัท อิ๊กดราซิล กรุ๊ป จำกัด (มหาชน),บริษัท ยัวซ่าแบตเตอรี่ ประเทศไทย จำกัด (มหาชน),บริษัท เซ็น คอร์ปอเรชั่น กรุ๊ป จำกัด (มหาชน),บริษัท ซิก้า อินโนเวชั่น จำกัด (มหาชน),บริษัทหลักทรัพย์ ซีมิโก้ จำกัด (มหาชน)


In [18]:
stocks = all_symbols.T
stocks

,symbol,exchange,industry,sector,name
2S,2S,MAI,สินค้าอุตสาหกรรม,-,บริษัท 2 เอส เมทัล จำกัด (มหาชน)
7UP,7UP,SET,ทรัพยากร,พลังงานและสาธารณูปโภค,บริษัท เซเว่น ยูทิลิตี้ส์ แอนด์ พาวเวอร์ จำกัด...
A,A,SET,อสังหาริมทรัพย์และก่อสร้าง,พัฒนาอสังหาริมทรัพย์,บริษัท อารียา พรอพเพอร์ตี้ จำกัด (มหาชน)
A5,A5,MAI,บริการ,-,บริษัท แอสเซท ไฟว์ กรุ๊ป จำกัด (มหาชน)
AAV,AAV,SET,บริการ,ขนส่งและโลจิสติกส์,บริษัท เอเชีย เอวิเอชั่น จำกัด (มหาชน)
...,...,...,...,...,...
YGG,YGG,MAI,บริการ,-,บริษัท อิ๊กดราซิล กรุ๊ป จำกัด (มหาชน)
YUASA,YUASA,MAI,สินค้าอุตสาหกรรม,-,บริษัท ยัวซ่าแบตเตอรี่ ประเทศไทย จำกัด (มหาชน)
ZEN,ZEN,SET,เกษตรและอุตสาหกรรมอาหาร,อาหารและเครื่องดื่ม,บริษัท เซ็น คอร์ปอเรชั่น กรุ๊ป จำกัด (มหาชน)
ZIGA,ZIGA,MAI,สินค้าอุตสาหกรรม,-,บริษัท ซิก้า อินโนเวชั่น จำกัด (มหาชน)


In [13]:
stocks.to_csv('all_symbols.csv')

In [20]:
SET_stocks = stocks[stocks['exchange']=='SET']
SET_stocks

,symbol,exchange,industry,sector,name
7UP,7UP,SET,ทรัพยากร,พลังงานและสาธารณูปโภค,บริษัท เซเว่น ยูทิลิตี้ส์ แอนด์ พาวเวอร์ จำกัด...
A,A,SET,อสังหาริมทรัพย์และก่อสร้าง,พัฒนาอสังหาริมทรัพย์,บริษัท อารียา พรอพเพอร์ตี้ จำกัด (มหาชน)
AAV,AAV,SET,บริการ,ขนส่งและโลจิสติกส์,บริษัท เอเชีย เอวิเอชั่น จำกัด (มหาชน)
ABPIF,ABPIF,SET,ทรัพยากร,พลังงานและสาธารณูปโภค,กองทุนรวมโครงสร้างพื้นฐานโรงไฟฟ้า อมตะ บี.กริม...
ACC,ACC,SET,สินค้าอุปโภคบริโภค,ของใช้ในครัวเรือนและสำนักงาน,บริษัท แอดวานซ์ คอนเนคชั่น คอร์ปอเรชั่น จำกัด ...
...,...,...,...,...,...
WP,WP,SET,ทรัพยากร,พลังงานและสาธารณูปโภค,บริษัท ดับบลิวพี เอ็นเนอร์ยี่ จำกัด (มหาชน)
WPH,WPH,SET,บริการ,การแพทย์,บริษัท โรงพยาบาลวัฒนแพทย์ ตรัง จำกัด (มหาชน)
YCI,YCI,SET,สินค้าอุตสาหกรรม,ปิโตรเคมีและเคมีภัณฑ์,บริษัท ยงไทย จำกัด (มหาชน)
ZEN,ZEN,SET,เกษตรและอุตสาหกรรมอาหาร,อาหารและเครื่องดื่ม,บริษัท เซ็น คอร์ปอเรชั่น กรุ๊ป จำกัด (มหาชน)


In [21]:
SET_stocks.to_csv('SET_symbols.csv')

In [ ]:
SET_stocks.to_csv('SET_symbols.csv')